In [ ]:
from stardist.models import StarDist3D
from pathlib import Path
import numpy as np
from tifffile import imread, imwrite
from csbdeep.utils import normalize

In [ ]:
DATA_PATH = (Path.home() / "Desktop/Code/CELLSEG_BENCHMARK/RESULTS/SUPERVISED_PERF_FIG/").resolve()
fold = 1
# pretrained_path = DATA_PATH / f"weights/fold_{fold}/cellpose/supervised_perf_fig.cellpose" 
pretrained_path = DATA_PATH / f"weights/fold_{fold}" 
assert pretrained_path.is_dir()
images_path = DATA_PATH / f"TRAINING/fold{fold}/IMAGES"
assert images_path.is_dir()
images_paths = list(images_path.glob("*tif"))

images_paths

In [ ]:
test_image = imread(images_paths[0])
n_channel = 1 if test_image.ndim == 3 else test_image.shape[-1]
axis_norm = (0,1,2)   # normalize channels independently
# axis_norm = (0,1,2,3) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))

In [ ]:
save_dir = DATA_PATH / f"TRAINING/fold{fold}/stardist"
if not save_dir.is_dir():
    save_dir.mkdir(parents=False, exist_ok=True)

In [ ]:
model = StarDist3D(None, name='stardist', basedir=pretrained_path)

NMS_threshold_values = np.arange(0.1, 1, 0.1)
prob_thresh_values = np.arange(0.1, 1, 0.1)
NMS_threshold_values = np.round(NMS_threshold_values, 2)
prob_thresh_values = np.round(prob_thresh_values, 2)

In [10]:
for image_p in images_paths:
    im_name = image_p.stem
    print(im_name)
    # for resample_val in resample:
    for prob_t in prob_thresh_values:
        for nms in NMS_threshold_values:
            img=imread(str(image_p))
            img = normalize(img, 1,99.8, axis=axis_norm)
            masks, details = model.predict_instances(img, prob_thresh=prob_t, nms_thresh=nms)
            save_name = str(save_dir / f"{im_name}_stardist_masks_nms_{nms}_prob_{prob_t}.tif")
            imwrite(save_name, masks)